# Histopathologic Cancer Detection

## Problem Description and Data Overview

I was tasked with creating an algorithm to identify metastatic cancer in small image patches taken from larger digital pathology scans. This is a binary image classification problem, where I needed to determine whether the center 32x32px region of a patch contains at least one pixel of tumor tissue.

### Dataset Description:

The dataset I used is a modified version of the PatchCamelyon (PCam) benchmark dataset.

- The images are 96x96px RGB patches.
- The center 32x32px region of each image determines the label.
- A positive label indicates the presence of at least one tumor pixel in the center region.
- The dataset does not contain duplicate images.

### Data Statistics:

- **Training set**: 220,025 images
- **Test set**: 57,458 images
- **Image format**: .tif files
- **Labels**: Binary (0 for non-tumor, 1 for tumor)

## Project Significance and Use Case

The goal of this project is to create a reliable model that can assist pathologists in detecting cancerous regions in histopathologic images. The significance lies in speeding up the diagnosis process, improving accuracy, and potentially saving lives by allowing for early detection of cancerous tissue. In real-world applications, this type of model could be deployed in hospitals to automate or assist in the screening of biopsies, reducing the workload on medical professionals and improving diagnostic throughput. With cancer being one of the leading causes of death worldwide, an automated detection system could have a huge impact on healthcare efficiency and patient outcomes. This type of deep learning solution could also be extended to various types of cancers, broadening its utility across different medical fields.

## Model Selection and Approach

Instead of relying solely on pre-established models like ResNet or EfficientNet, I decided to focus on designing a **custom CNN model** for this problem. While pre-trained models have been successful across various domains, this project gave me an opportunity to see how well I could do by building a model specifically tailored to the characteristics of my dataset. Custom CNN architectures provide the flexibility to adjust hyperparameters like the number of layers, filter sizes, and dropout rates, which could lead to better optimization for my specific task. By doing this, I aimed to compare the performance of my custom CNN to that of widely recognized models, like ResNet and EfficientNet, and understand how a carefully designed model can compete with established architectures. 

This approach not only allowed me to better understand the intricacies of model design but also gave me greater control over the feature extraction process, allowing for a more specialized solution for detecting cancer in histopathologic images.

### System Setup and Configuration

This code block handles the essential setup and configuration for the deep learning environment. It performs the following tasks:

1. **Imports**: Loads necessary libraries for data manipulation, visualization, image processing, machine learning, and deep learning (TensorFlow, Keras).
2. **GPU Configuration**: Configures available GPUs for optimized memory usage and enables mixed precision training for faster computation with reduced memory footprint.
3. **System Checks**: Verifies if TensorFlow is built with CUDA, checks the number of available GPUs, and outputs the versions of CUDA and cuDNN in use.
4. **Paths and Data Loading**: Defines file paths for training and test data, and loads the training labels from a CSV file for further processing.

This setup ensures the environment is optimized for efficient deep learning model training and testing.


In [ ]:
# Standard Library Imports
import os
import warnings

# Data Manipulation and Analysis
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Image Processing
from skimage import io, filters, exposure
from skimage.color import rgb2gray

# Machine Learning
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, precision_recall_curve

# Deep Learning - TensorFlow and Keras
import tensorflow as tf
from tensorflow.keras import layers, models, mixed_precision, backend as K
from tensorflow.keras.applications import ResNet50, EfficientNetB0
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.sysconfig as sysconfig

# TensorFlow Preprocessing Functions for Models
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

# Hyperparameter Tuning
import keras_tuner as kt

# Progress Bar
from tqdm import tqdm

# Configurations
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
warnings.simplefilter('ignore', UserWarning)

# GPU and mixed precision setup
tf.config.optimizer.set_jit(True)  # Enable XLA (Accelerated Linear Algebra optimization)

# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print("Error while setting memory growth:", e)

# Set global mixed precision policy
mixed_precision.set_global_policy('mixed_float16')
print("Mixed precision policy set to:", mixed_precision.global_policy().name)

# Check available devices
print("Available devices:", tf.config.list_logical_devices())

# Check if TensorFlow is built with CUDA
print("Is TensorFlow built with CUDA:", tf.test.is_built_with_cuda())

# Check number of available GPUs
print("Num GPUs Available:", len(gpus))

# Check CUDA and cuDNN versions being used by TensorFlow
cuda_version = sysconfig.get_build_info().get("cuda_version", "Unknown")
cudnn_version = sysconfig.get_build_info().get("cudnn_version", "Unknown")

print("CUDA version:", cuda_version)
print("cuDNN version:", cudnn_version)

# Set paths
TRAIN_PATH = 'train/'
TEST_PATH = 'test/'
TRAIN_LABELS_PATH = 'train_labels.csv'

# Load labels
train_labels = pd.read_csv(TRAIN_LABELS_PATH)

## Exploratory Data Analysis (EDA)

In [ ]:
# Display first few rows
print("First few rows of training labels:")
print(train_labels.head())

# Check class distribution
class_distribution = train_labels['label'].value_counts(normalize=True)
print("\nClass Distribution:")
print(class_distribution)

# Visualize class distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='label', data=train_labels)
plt.title('Class Distribution')
plt.show()

# Validate image file paths
def validate_image_paths(dataframe, image_dir, file_extension='.tif'):
    missing_files = []
    valid_files = []
    for _, row in dataframe.iterrows():
        image_filename = row['id'] + file_extension
        image_path = os.path.join(image_dir, image_filename)
        if not os.path.isfile(image_path):
            missing_files.append(image_filename)
        else:
            valid_files.append(image_filename)
    
    if missing_files:
        print(f"\n{len(missing_files)} image files listed in the CSV are missing in the directory: {image_dir}")
        print("Missing files example:", missing_files[:5])
    else:
        print("All image files listed in the CSV are present in the directory.")
    
    return valid_files

# Validate training image paths
valid_train_files = validate_image_paths(train_labels, TRAIN_PATH)
# Verify if train_data contains only valid files
train_labels['valid'] = train_labels['id'] + '.tif'
train_labels = train_labels[train_labels['valid'].isin(valid_train_files)].drop('valid', axis=1)

# Load and display some sample images
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
for i, (idx, row) in enumerate(train_labels.sample(10).iterrows()):
    # Adjust image path to include .tif extension
    img_path = os.path.join(TRAIN_PATH, row['id'] + '.tif')
    if os.path.exists(img_path):
        img = io.imread(img_path)
        ax = axes[i//5, i%5]
        ax.imshow(img)
        ax.set_title(f"Label: {row['label']}")
        ax.axis('off')
plt.tight_layout()
plt.show()

# Analyze image properties
img_sizes = []
img_channels = []

print("Analyzing image properties for a sample of 1000 images...")
for img_name in tqdm(os.listdir(TRAIN_PATH)[:1000]):  # Sample 1000 images
    img_path = os.path.join(TRAIN_PATH, img_name)
    if os.path.exists(img_path):
        img = io.imread(img_path)
        img_sizes.append(img.shape[:2])
        img_channels.append(img.shape[2] if len(img.shape) > 2 else 1)

print("\nImage Sizes Distribution:")
print(pd.Series(img_sizes).value_counts())
print("\nImage Channels Distribution:")
print(pd.Series(img_channels).value_counts())

# Analyze pixel intensity distribution for sample images
print("Analyzing pixel intensity distribution for sample images...")
sample_images = []
for img_name in os.listdir(TRAIN_PATH)[:1000]:
    img_path = os.path.join(TRAIN_PATH, img_name)
    if os.path.exists(img_path):
        sample_images.append(io.imread(img_path))

pixel_intensities = np.concatenate([img.ravel() for img in sample_images])

plt.figure(figsize=(10, 6))
sns.histplot(pixel_intensities, kde=True, bins=50)
plt.title('Pixel Intensity Distribution')
plt.xlabel('Pixel Intensity')
plt.show()

# Color analysis (for RGB channels)
avg_colors = []

for img_name in tqdm(os.listdir(TRAIN_PATH)[:1000]):  # Adjust sample size
    img_path = os.path.join(TRAIN_PATH, img_name)
    if os.path.exists(img_path):
        img = io.imread(img_path)
        if len(img.shape) == 3:  # Only process RGB images
            avg_color_per_channel = img.mean(axis=(0, 1))
            avg_colors.append(avg_color_per_channel)

# Convert to numpy array
avg_colors = np.array(avg_colors)

# Plot the average color distribution for each channel
plt.figure(figsize=(10, 6))
plt.hist(avg_colors[:, 0], bins=50, color='red', alpha=0.6, label='Red')
plt.hist(avg_colors[:, 1], bins=50, color='green', alpha=0.6, label='Green')
plt.hist(avg_colors[:, 2], bins=50, color='blue', alpha=0.6, label='Blue')
plt.title('Average Color Distribution Across Channels')
plt.xlabel('Average Pixel Value')
plt.ylabel('Count')
plt.legend()
plt.show()

# Edge detection example using Sobel filter
edges = []

for img_name in tqdm(os.listdir(TRAIN_PATH)[:100]):  # Sample of 100 images
    img_path = os.path.join(TRAIN_PATH, img_name)
    if os.path.exists(img_path):
        img = io.imread(img_path)
        if len(img.shape) == 3:  # Convert to grayscale for edge detection
            img_gray = rgb2gray(img)
        else:
            img_gray = img
        edge_sobel = filters.sobel(img_gray)
        edges.append(edge_sobel)

# Display some edge-detected images
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
for i, ax in enumerate(axes.flat):
    ax.imshow(edges[i], cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.show()

### EDA Findings:

1. **Class distribution**:
   - The dataset is slightly imbalanced:
     - Class 0 (non-tumor): 59.50% of the samples
     - Class 1 (tumor): 40.50% of the samples
   - This imbalance is not severe, but it’s worth considering in our modeling approach.

   ![Class Distribution](visualizations/Class_Distribution.png)

2. **Image properties**:
   - All images have a consistent size of 96x96 pixels.
   - All images are in RGB format (3 channels).

   ![Sample Images](visualizations/Sample_Images.png)

3. **Pixel intensity distribution**:
   - The pixel intensity distribution is right-skewed, with a peak near the higher intensity values (200-250).
   - There’s also a smaller peak at lower intensities (0-50), likely representing darker regions in the images.
   - This suggests good contrast in the images, which could be helpful for distinguishing features.

   ![Pixel Intensity Distribution](visualizations/Pixel_Intensity_Distribution.png)

4. **Color distribution across channels**:
   - The average color distribution shows differences in pixel values for the red, green, and blue channels. This variance in color information could be leveraged during training for better feature extraction.

   ![Color Distribution](visualizations/color_distribution.png)

5. **Edge detection**:
   - I explored edge detection techniques (using the Sobel filter) to highlight the boundaries within the tissue samples. However, edge detection didn’t reveal significant additional information for distinguishing between the two classes, and I decided not to pursue it further.

   ![Edge Detection](visualizations/edge.png)


### Analysis Plan:

Based on these findings, my updated analysis plan will include:

1. **Data augmentation** to address the slight class imbalance. Techniques such as random rotations, flips, and shifts will be applied.
2. **Image preprocessing** using the cropping and super-resolution technique. Images are cropped to 32x32 and then resized to 224x224 using the ESRGAN super-resolution model.
3. **Pixel normalization** to improve model convergence. Given the skewed pixel intensity distribution, normalization techniques like histogram equalization might be beneficial.
4. **CNN architecture** optimized for 224x224 RGB images, based on the consistent image size after preprocessing.
5. **Stratified k-fold cross-validation** to ensure representative class distribution across folds.
6. **Attention mechanisms** or similar techniques will be explored to focus on areas of dense cell distribution, which seem indicative of tumor presence.
7. **Transfer learning** using pre-trained models such as ResNet or EfficientNetB0, leveraging their ability to process RGB images with the given resolution.
8. **Learning rate scheduling and early stopping** to manage training dynamics carefully, given the subtle differences between the two classes.

### Image Preprocessing with Super-Resolution and Cropping

In the dataset provided, each image is a 96x96 pixel RGB patch, and the critical information used to determine whether a tumor is present lies in the **center 32x32 region**. Specifically, the tumor label is defined by whether or not there is at least **one tumor pixel in that 32x32 region**. Thus, the information that is most relevant for classification resides in a relatively small portion of the overall image. 

To leverage this, I implemented a smart preprocessing approach that focuses on cropping the center 32x32 region of each image and applying **super-resolution techniques** to increase the resolution to 224x224. This approach has several advantages:

#### Key Steps:

1. **Center Cropping**: 
   - I first crop the **32x32 pixel region** from the center of each 96x96 image.
   - This ensures that I am focusing only on the region that is critical for classification (i.e., the area where tumor pixels are present or absent).
   
2. **Super-Resolution**: 
   - After cropping, the 32x32 pixel region is passed through an **ESRGAN** (Enhanced Super-Resolution Generative Adversarial Network) model, which increases the resolution of the image to **224x224**.
   - By doing this, I allow our deep learning models to have **better access to fine-grained details** that may otherwise be lost in a low-resolution image, such as subtle variations in cell structures or tissue textures.
   
3. **Resizing**:
   - After applying super-resolution, the images are resized using a **LANCZOS filter**, ensuring smooth scaling to a final resolution of **224x224**.

#### Why This Approach is Effective:

1. **Focusing on the Relevant Region**:
   - By cropping the center 32x32 region, I eliminate extraneous information that is not relevant to the task (i.e., tissue areas that do not impact the classification).
   - This ensures that the models are only trained on **the most critical region** of each image, allowing them to learn more effectively.

2. **Enhancing Visual Detail**:
   - The center region, although crucial, is originally very low in resolution (32x32). If used directly, it may not provide sufficient detail for models to accurately classify the presence of tumors.
   - **Super-resolution** techniques help to restore some of the lost details in the original image, allowing models to make more informed decisions. Features that may be blurry or unclear in the low-resolution 32x32 image become sharper and easier for the model to detect in the super-resolved version.
   
3. **Improving Model Performance**:
   - Deep learning models, particularly those used in computer vision, are often optimized for higher-resolution images. By resizing to 224x224, I ensure that the input matches the typical input sizes of common architectures (e.g., ResNet, EfficientNet).
   - Higher resolution input also allows models to detect **subtle patterns** and textures that may not be discernible in smaller, lower-resolution images.

4. **Efficient Use of GPU Resources**:
   - By using a **super-resolution model** (ESRGAN) in combination with GPU resources, I was able to efficiently process thousands of images and prepare them for training.
   - This allowed me to strike a balance between **performance** (processing time) and **accuracy** (image quality).

In [ ]:
'''
def load_super_resolution_model():
    # Load the ESRGAN model from TensorFlow Hub
    model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
    return model


def super_resolve_image(image, model):
    # Convert PIL Image to Tensor
    lr_image = tf.convert_to_tensor(np.array(image), dtype=tf.float32)

    # Run the model
    sr_image = model(tf.expand_dims(lr_image, axis=0))

    # Remove batch dimension
    sr_image = tf.squeeze(sr_image)

    # Convert to uint8
    sr_image = tf.clip_by_value(sr_image, 0, 255)
    sr_image = tf.cast(sr_image, tf.uint8)

    # Convert Tensor to PIL Image
    sr_image = Image.fromarray(sr_image.numpy())

    # Resize to exactly 224x224 using LANCZOS filter
    sr_image = sr_image.resize((224, 224), Image.LANCZOS)

    return sr_image


def preprocess_image(image):
    # Crop the center 32x32 region
    crop_size = 32
    height, width, _ = image.shape
    start_x = (width - crop_size) // 2
    start_y = (height - crop_size) // 2
    image = image[start_y:start_y + crop_size, start_x:start_x + crop_size]

    # Convert to uint8 if not already
    image = image.astype(np.uint8)
    return image


def preprocess_images(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_filenames = [f for f in os.listdir(input_dir) if f.endswith('.tif')]
    print(f"Found {len(image_filenames)} images to process.")

    # Load the super-resolution model
    sr_model = load_super_resolution_model()

    # Use GPU if available
    device = '/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'

    for filename in tqdm(image_filenames, desc="Processing images"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        # Read image using PIL
        image = Image.open(input_path).convert('RGB')

        # Preprocess the image
        image = np.array(image)
        processed_image = preprocess_image(image)

        # Convert back to PIL Image
        processed_image = Image.fromarray(processed_image)

        # Super-resolve the image within the device context
        with tf.device(device):
            sr_image = super_resolve_image(processed_image, sr_model)

        # Save the processed image
        sr_image.save(output_path)

    print("Processing completed.")

input_directory = sys.argv[1]
output_directory = sys.argv[2]

preprocess_images(input_directory, output_directory)
'''

### Visualization of the Transformation:

In the image comparison below, you can see the stark difference between the **original image** (left) and the **super-resolved preprocessed image** (right):

![Original vs Preprocessed Image](visualizations/original_vs_preprocessed_image.png)

#### Summary:

This preprocessing approach allows our models to train on higher-quality images, while focusing on the regions most relevant for classification. By combining **cropping** and **super-resolution**, I improve the **signal-to-noise ratio** in the training data, which ultimately leads to better model performance. This strategy proved to be a key factor in enhancing the **accuracy** and **generalizability** of our models during training.

## Model Architecture
For this task, I will implement and compare multiple CNN architectures, taking into account our EDA findings. We'll start with a custom CNN designed for our specific image properties, and then implement transfer learning using pre-trained models like ResNet50 and EfficientNetB0.
### Custom CNN

In [ ]:
def create_custom_cnn(input_shape=(224, 224, 3), name='CustomCNN_Model'):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name=name)
    return model

# Instantiate models to view their summaries
custom_cnn_model = create_custom_cnn()
print("Custom CNN Model Summary:")
custom_cnn_model.summary()

### Custom CNN Model Summary

| Layer (type)                    | Output Shape           | Param #       |
|----------------------------------|------------------------|---------------|
| **input_layer (InputLayer)**     | (None, 224, 224, 3)    | 0             |
| **cast (Cast)**                  | (None, 224, 224, 3)    | 0             |
| **conv2d (Conv2D)**              | (None, 222, 222, 32)   | 896           |
| **batch_normalization**          | (None, 222, 222, 32)   | 128           |
| **max_pooling2d (MaxPooling2D)** | (None, 111, 111, 32)   | 0             |
| **conv2d_1 (Conv2D)**            | (None, 109, 109, 64)   | 18,496        |
| **batch_normalization_1**        | (None, 109, 109, 64)   | 256           |
| **max_pooling2d_1 (MaxPooling2D)**| (None, 54, 54, 64)    | 0             |
| **conv2d_2 (Conv2D)**            | (None, 52, 52, 128)    | 73,856        |
| **batch_normalization_2**        | (None, 52, 52, 128)    | 512           |
| **global_average_pooling2d**     | (None, 128)            | 0             |
| **dense (Dense)**                | (None, 1024)           | 132,096       |
| **dropout (Dropout)**            | (None, 1024)           | 0             |
| **dense_1 (Dense)**              | (None, 1)              | 1,025         |

**Total params**: 227,265 (887.75 KB)  
**Trainable params**: 226,817 (886.00 KB)  
**Non-trainable params**: 448 (1.75 KB)


### Transfer Learning with ResNet50

In [ ]:
def create_resnet50_model(input_shape=(224, 224, 3), name='ResNet50_Model'):
    base_model = resnet.ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=base_model.input, outputs=output, name=name)
    base_model.trainable = False  # Freeze the base model layers
    return model

resnet50_model = create_resnet50_model()
print("\nResNet50 Model Summary:")
resnet50_model.summary()

### ResNet50 Model Summary

| Layer (type)            | Output Shape         | Param #      | Connected to         |
|-------------------------|----------------------|--------------|----------------------|
| **input_layer_1**        | (None, 224, 224, 3)  | 0            | -                    |
| **cast_1 (Cast)**        | (None, 224, 224, 3)  | 0            | input_layer_1[0][0]  |
| **conv1_pad (ZeroPadding2D)**| (None, 230, 230, 3)| 0          | cast_1[0][0]         |
| **conv1_conv (Conv2D)**  | (None, 112, 112, 64) | 9,472        | conv1_pad[0][0]      |
| **conv1_bn (BatchNormalization)** | (None, 112, 112, 64) | 256    | conv1_conv[0][0]     |
| **conv1_relu (Activation)** | (None, 112, 112, 64) | 0         | conv1_bn[0][0]       |
| **pool1_pad (ZeroPadding2D)** | (None, 114, 114, 64) | 0        | conv1_relu[0][0]     |
| **pool1_pool (MaxPooling2D)** | (None, 56, 56, 64) | 0         | pool1_pad[0][0]      |
| **conv2_block1_1_conv (Conv2D)** | (None, 56, 56, 64) | 4,160   | pool1_pool[0][0]     |
| **conv2_block1_1_bn (BatchNormalization)** | (None, 56, 56, 64) | 256   | conv2_block1_1_conv[0][0] |
| **conv2_block1_1_relu (Activation)** | (None, 56, 56, 64) | 0     | conv2_block1_1_bn[0][0]   |
| **conv2_block1_2_conv (Conv2D)** | (None, 56, 56, 64) | 36,928  | conv2_block1_1_relu[0][0] |
| **conv2_block1_2_bn (BatchNormalization)** | (None, 56, 56, 64) | 256   | conv2_block1_2_conv[0][0] |
| **conv2_block1_2_relu (Activation)** | (None, 56, 56, 64) | 0     | conv2_block1_2_bn[0][0]   |
| **conv2_block1_0_conv (Conv2D)** | (None, 56, 56, 256) | 16,640 | pool1_pool[0][0]      |
| **conv2_block1_3_conv (Conv2D)** | (None, 56, 56, 256) | 16,640 | conv2_block1_2_relu[0][0] |
| **conv2_block1_0_bn (BatchNormalization)** | (None, 56, 56, 256) | 1,024  | conv2_block1_0_conv[0][0] |
| **conv2_block1_3_bn (BatchNormalization)** | (None, 56, 56, 256) | 1,024  | conv2_block1_3_conv[0][0] |
| **conv2_block1_add (Add)** | (None, 56, 56, 256) | 0           | conv2_block1_0_bn[0][0], conv2_block1_3_bn[0][0] |
| **conv2_block1_out (Activation)** | (None, 56, 56, 256) | 0       | conv2_block1_add[0][0] |

*... (and so on for other layers)*

**Total params**: 25,686,913 (97.99 MB)  
**Trainable params**: 2,099,201 (8.01 MB)  
**Non-trainable params**: 23,587,712 (89.98 MB)


### Transfer Learning with EfficientNetB0

In [ ]:
def create_efficientnet_model(input_shape=(224, 224, 3), name='EfficientNetB0_Model'):
    base_model = efficientnet.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = models.Model(inputs=base_model.input, outputs=output, name=name)
    base_model.trainable = False  # Freeze the base model layers
    return model

efficientnet_model = create_efficientnet_model()
print("\nEfficientNetB0 Model Summary:")
efficientnet_model.summary()

### EfficientNetB0 Model Summary

| Layer (type)           | Output Shape        | Param #    | Connected to          |
|------------------------|---------------------|------------|-----------------------|
| **input_layer_2**       | (None, 224, 224, 3) | 0          | -                     |
| **cast_2 (Cast)**       | (None, 224, 224, 3) | 0          | input_layer_2[0][0]   |
| **rescaling**           | (None, 224, 224, 3) | 0          | cast_2[0][0]          |
| **normalization**       | (None, 224, 224, 3) | 7          | rescaling[0][0]       |
| **rescaling_1**         | (None, 224, 224, 3) | 0          | normalization[0][0]   |
| **stem_conv_pad**       | (None, 225, 225, 3) | 0          | rescaling_1[0][0]     |
| **stem_conv (Conv2D)**  | (None, 112, 112, 32)| 864        | stem_conv_pad[0][0]   |
| **stem_bn (BatchNormalization)** | (None, 112, 112, 32)| 128   | stem_conv[0][0]       |
| **stem_activation (Activation)** | (None, 112, 112, 32) | 0     | stem_bn[0][0]         |
| **block1a_dwconv (DepthwiseConv2D)** | (None, 112, 112, 32) | 288  | stem_activation[0][0] |
| **block1a_bn (BatchNormalization)** | (None, 112, 112, 32) | 128   | block1a_dwconv[0][0]  |
| **block1a_activation (Activation)** | (None, 112, 112, 32) | 0    | block1a_bn[0][0]      |
| **block1a_se_squeeze (GlobalAveragePooling2D)** | (None, 32) | 0  | block1a_activation[0][0] |
| **block1a_se_reshape (Reshape)** | (None, 1, 1, 32) | 0        | block1a_se_squeeze[0][0] |
| **block1a_se_reduce (Conv2D)** | (None, 1, 1, 8) | 264        | block1a_se_reshape[0][0] |
| **block1a_se_expand (Conv2D)** | (None, 1, 1, 32) | 288       | block1a_se_reduce[0][0] |
| **block1a_se_excite (Multiply)** | (None, 112, 112, 32) | 0     | block1a_activation[0][0], block1a_se_expand[0][0] |
| **block1a_project_conv (Conv2D)** | (None, 112, 112, 16) | 512  | block1a_se_excite[0][0] |
| **block1a_project_bn (BatchNormalization)** | (None, 112, 112, 16) | 64 | block1a_project_conv[0][0] |
| **block2a_expand_conv (Conv2D)** | (None, 112, 112, 96) | 1,536 | block1a_project_bn[0][0] |
| **block2a_expand_bn (BatchNormalization)** | (None, 112, 112, 96) | 384 | block2a_expand_conv[0][0] |
| **block2a_expand_activation (Activation)** | (None, 112, 112, 96) | 0 | block2a_expand_bn[0][0] |
| **block2a_dwconv_pad (ZeroPadding2D)** | (None, 113, 113, 96) | 0 | block2a_expand_activation[0][0] |
| **block2a_dwconv (DepthwiseConv2D)** | (None, 56, 56, 96) | 864 | block2a_dwconv_pad[0][0] |
| **block2a_bn (BatchNormalization)** | (None, 56, 56, 96) | 384 | block2a_dwconv[0][0] |
| **block2a_activation (Activation)** | (None, 56, 56, 96) | 0 | block2a_bn[0][0] |
| **block2a_se_squeeze (GlobalAveragePooling2D)** | (None, 96) | 0 | block2a_activation[0][0] |
| **block2a_se_reshape (Reshape)** | (None, 1, 1, 96) | 0 | block2a_se_squeeze[0][0] |
| **block2a_se_reduce (Conv2D)** | (None, 1, 1, 4) | 388 | block2a_se_reshape[0][0] |
| **block2a_se_expand (Conv2D)** | (None, 1, 1, 96) | 480 | block2a_se_reduce[0][0] |
| **block2a_se_excite (Multiply)** | (None, 56, 56, 96) | 0 | block2a_activation[0][0], block2a_se_expand[0][0] |
| **block2a_project_conv (Conv2D)** | (None, 56, 56, 24) | 2,304 | block2a_se_excite[0][0] |
| **block2a_project_bn (BatchNormalization)** | (None, 56, 56, 24) | 96 | block2a_project_conv[0][0] |
| **block2b_expand_conv (Conv2D)** | (None, 56, 56, 144) | 3,456 | block2a_project_bn[0][0] |
| ... (and so on for other layers) |
| **global_average_pooling2d** | (None, 1280) | 0 | top_activation[0][0] |
| **dense_4 (Dense)** | (None, 1024) | 1,311,744 | global_average_pooling2d[0][0] |
| **dropout_2 (Dropout)** | (None, 1024) | 0 | dense_4[0][0] |
| **dense_5 (Dense)** | (None, 1) | 1,025 | dropout_2[0][0] |
| **Total params**: 5,362,340 (20.46 MB) |
| **Trainable params**: 1,312,769 (5.01 MB) |
| **Non-trainable params**: 4,049,571 (15.45 MB) |


These architectures were chosen and modified for the following reasons:

1. Custom CNN (Total params: 61,185 - 239.00 KB): 
   - Establishes a baseline performance.
   - Added BatchNormalization layers to help with the varying pixel intensities we observed.
   - Used GlobalAveragePooling2D to reduce parameters and maintain spatial information.
   - Added Dropout for regularization, considering the relatively small dataset.

2. ResNet50 (Total params: 25,686,913 - 97.99 MB): 
   - Known for its ability to train very deep networks effectively.
   - The residual connections can help in capturing fine-grained details in histopathology images.

3. EfficientNetB0 (Total params: 5,362,340 - 20.46 MB): 
   - Offers a good balance between model size and performance.
   - Particularly efficient for smaller image sizes like our 96x96 pixels.

For all models:
- I am using a sigmoid activation for the final layer, suitable for our binary classification task.
- I added Dropout layers in the fully connected parts of the networks to prevent overfitting.
- The input shape is set to (96, 96, 3) to match our image properties.
- For transfer learning models (ResNet50 and EfficientNetB0), we're initially freezing the base layers to leverage pre-trained weights, as evidenced by the large number of non-trainable parameters.

The significant difference in model sizes highlights the trade-off between model complexity and potential performance, with our custom CNN being the smallest and ResNet50 being the largest.

In the training phase, we'll implement data augmentation to address the slight class imbalance and increase model robustness. We'll also use callbacks for learning rate adjustment and early stopping to fine-tune the training process.

## Model Training and Cross-Validation Comparison 
I utilize the ImageDataGenerator from Keras for data augmentation, normalization, and loading of training and validation images. The generators apply augmentation techniques such as random rotation, shifting, shearing, and flipping to the training data, helping to improve model generalization and reduce overfitting.

The models are evaluated using Stratified K-Fold Cross-Validation with n_splits=5 to ensure robust evaluation. This method maintains the proportion of class labels in each fold, accounting for the slight class imbalance present in the dataset. During each fold, I train a new model instance, validate on the corresponding validation set, and calculate evaluation metrics such as accuracy, precision, recall, and F1-score for a comprehensive assessment.

The following code block performs the training and evaluation of the models using cross-validation, plots the learning curves, and displays the results for comparison.

In [ ]:
# Define preprocessing functions
def custom_cnn_preprocess(image):
    # Normalize pixel values to [0, 1]
    image = image.astype(np.float32) / 255.0
    return image

# Update the preprocess_for_resnet function
def preprocess_for_resnet(image):
    image = image.astype(np.float32)
    image = resnet_preprocess(image)
    return image

# Update the preprocess_for_efficientnet function
def preprocess_for_efficientnet(image):
    image = image.astype(np.float32)
    image = efficientnet_preprocess(image)
    return image

def create_generators(train_data, val_data, train_path, batch_size, target_size=(224, 224), preprocess_func=None):
    train_data = train_data.copy()
    val_data = val_data.copy()
    
    # Ensure filenames end with '.tif'
    train_data['id'] = train_data['id'].apply(lambda x: x if x.endswith('.tif') else x + '.tif')
    val_data['id'] = val_data['id'].apply(lambda x: x if x.endswith('.tif') else x + '.tif')
    
    # Create ImageDataGenerator for training data with augmentations
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_func,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='constant',
        cval=200,
        brightness_range=(0.9, 1.1),
        channel_shift_range=20
    )

    # Create generator for training data
    train_generator = train_datagen.flow_from_dataframe(
        train_data,
        directory=train_path,
        x_col='id',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    # Create ImageDataGenerator for validation data with no augmentation
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_func)
    val_generator = val_datagen.flow_from_dataframe(
        val_data,
        directory=train_path,
        x_col='id',
        y_col='label',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    
    # Debug: Inspect a single batch
    x_train_batch, y_train_batch = next(train_generator)
    print(f"Training Batch - Inputs shape: {x_train_batch.shape}, Labels shape: {y_train_batch.shape}")
    print(f"Training Batch - Inputs dtype: {x_train_batch.dtype}, Labels dtype: {y_train_batch.dtype}")
    
    x_val_batch, y_val_batch = next(val_generator)
    print(f"Validation Batch - Inputs shape: {x_val_batch.shape}, Labels shape: {y_val_batch.shape}")
    print(f"Validation Batch - Inputs dtype: {x_val_batch.dtype}, Labels dtype: {y_val_batch.dtype}")
    
    return train_generator, val_generator, len(train_data), len(val_data)


def compile_model(model, learning_rate, loss_function='binary_crossentropy'):
    optimizer = Adam(learning_rate=learning_rate)
    
    # Use passed-in loss function
    loss = loss_function
       
    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=[
            'accuracy',
            tf.keras.metrics.AUC(name='auc'),
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall')
        ]
    )
    return model

# Update the train_model function
def train_model(model, train_generator, val_generator, learning_rate, train_samples, val_samples, batch_size=128, epochs=5):
    print("Model input shape:", model.input_shape)

    # Always compile the model to ensure it's set up correctly
    model = compile_model(model, learning_rate)

    callbacks = [
        EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6),
        tf.keras.callbacks.ModelCheckpoint(
            f"best_model_{model.name}.keras", save_best_only=True, monitor='val_accuracy'
        ),
    ]

    # Fit model without manually resetting steps_per_epoch
    history = model.fit(
        train_generator,
        epochs=epochs,
        validation_data=val_generator,
        callbacks=callbacks,
        verbose=1
    )

    return history, model


# Function to get a fresh copy of a model after each fold
def get_fresh_model(model_fn):
    # Re-initialize the model by calling the model creation function
    model = model_fn()
    return model

# Define a function to evaluate model performance with additional metrics
def evaluate_model_performance(model, data_generator):
    # Get predictions and true labels
    data_generator.reset()
    y_true = []
    y_pred_proba = []
    
    for i in range(len(data_generator)):
        x, y = data_generator[i]
        y_true.extend(y)
        y_pred_proba.extend(model.predict(x).flatten())
    
    y_true = np.array(y_true).astype(int)  # Ensure labels are integers
    y_pred_proba = np.array(y_pred_proba)
    
    # Determine the optimal threshold
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)  # Avoid division by zero
    optimal_idx = np.argmax(f1_scores)
    optimal_threshold = thresholds[optimal_idx]
    print(f"Optimal Threshold: {optimal_threshold}")
    
    # Apply optimal threshold
    y_pred = (y_pred_proba >= optimal_threshold).astype(int)
    
    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision_metric = precision_score(y_true, y_pred)
    recall_metric = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred_proba)
    cm = confusion_matrix(y_true, y_pred)
    
    # Print results
    print("\nModel Performance Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision_metric:.4f}")
    print(f"Recall: {recall_metric:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    return {
        'accuracy': accuracy,
        'precision': precision_metric,
        'recall': recall_metric,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'confusion_matrix': cm
    }

# Define a function to plot learning curves from the history
def plot_learning_curves(history, model_name):
    metrics = ['loss', 'accuracy', 'auc', 'precision', 'recall']
    fig, axes = plt.subplots(3, 2, figsize=(15, 20))
    axes = axes.flatten()

    for i, metric in enumerate(metrics):
        if metric in history.history:
            axes[i].plot(history.history[metric], label=f'Train {metric}')
            axes[i].plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
            axes[i].set_title(f'{model_name} {metric.capitalize()}')
            axes[i].set_xlabel('Epoch')
            axes[i].set_ylabel(metric.capitalize())
            axes[i].legend()

    plt.tight_layout()
    plt.show()

# Update the model_dict to use the new preprocessing functions
model_dict = {
    'Custom CNN': (create_custom_cnn, custom_cnn_preprocess, 0.0001),  # Reduced learning rate
    'ResNet50': (create_resnet50_model, preprocess_for_resnet, 0.00001),
    'EfficientNetB0': (create_efficientnet_model, preprocess_for_efficientnet, 0.0001)
}

# Use updated preprocessed train images
PREPROCESSED_TRAIN_PATH = 'train_preprocessed/'

# Perform train/test split
train_data, test_data = train_test_split(train_labels, test_size=0.2, stratify=train_labels['label'], random_state=42)

print(f"Train set size: {len(train_data)}")
print(f"Test set size: {len(test_data)}")

# Train and evaluate each model using cross-validation
results = {}  # Store results for each model
batch_size = 128  # Set batch_size globally

# Perform Stratified K-Fold Cross-validation
n_splits = 2
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Main Loop
for model_name, (model_fn, preprocess_func, learning_rate) in model_dict.items():
    print(f"\nTraining {model_name} with learning rate: {learning_rate}")
    fold_scores = []
    history_all_folds = []  # Store history for all folds

    for fold, (train_idx, val_idx) in enumerate(skf.split(train_data['id'], train_data['label']), 1):
        print(f"Fold {fold}")

        # Split training and validation data for the current fold
        fold_train_data = train_data.iloc[train_idx].reset_index(drop=True)
        fold_val_data = train_data.iloc[val_idx].reset_index(drop=True)

        # Convert labels to strings here
        fold_train_data['label'] = fold_train_data['label'].astype(str)
        fold_val_data['label'] = fold_val_data['label'].astype(str)
        
        # Create a fresh copy of the model for this fold
        model = get_fresh_model(model_fn)

        # Create data generators
        train_generator, val_generator, train_samples, val_samples = create_generators(
            fold_train_data, fold_val_data, PREPROCESSED_TRAIN_PATH,
            batch_size=batch_size,
            target_size=(224, 224),
            preprocess_func=preprocess_func
        )
    
        # Train the model
        history, trained_model = train_model(
            model,
            train_generator,
            val_generator,
            learning_rate,
            train_samples,
            val_samples,
            batch_size=batch_size,
            epochs=2
        )

        # Save the history
        history_all_folds.append(history)

        # Evaluation code
        print(f"\nEvaluation for {model_name}, Fold {fold}:")

        print("Training Metrics from the last epoch:")
        print(f"Accuracy: {history.history['accuracy'][-1]:.4f}")
        print(f"AUC: {history.history['auc'][-1]:.4f}")
        print(f"Loss: {history.history['loss'][-1]:.4f}")
        print(f"Precision: {history.history['precision'][-1]:.4f}")
        print(f"Recall: {history.history['recall'][-1]:.4f}")

        print("\nValidation Metrics:")
        val_results = trained_model.evaluate(val_generator)
        for metric, value in zip(trained_model.metrics_names, val_results):
            print(f"{metric}: {value:.4f}")

        print("\nCustom Evaluation Metrics:")
        custom_metrics = evaluate_model_performance(trained_model, val_generator)

        # Append the custom evaluation accuracy to fold_scores
        fold_scores.append(custom_metrics['accuracy'])

    # Calculate and display the mean and standard deviation of accuracy scores for the model
    mean_acc = np.mean(fold_scores)
    std_acc = np.std(fold_scores)
    results[model_name] = {'mean_acc': mean_acc, 'std_acc': std_acc}
    print(f"{model_name} - Mean Accuracy: {mean_acc:.4f} (+/- {std_acc:.4f})")

    # Plot the learning curves for the last fold
    plot_learning_curves(history_all_folds[-1], model_name)

# Plot the results for model comparison
plt.figure(figsize=(12, 6))
for model_name, scores in results.items():
    plt.bar(model_name, scores['mean_acc'], yerr=scores['std_acc'], capsize=10)
plt.title('Model Comparison - Mean Accuracy Scores')
plt.ylabel('Mean Accuracy')
plt.ylim(0.5, 1.0)
plt.show()

# Identify and print the best-performing model based on mean accuracy score
best_model_name = max(results, key=lambda key: results[key]['mean_acc'])
best_model, best_preprocess_func, initial_learning_rate = model_dict[best_model_name]
print(f"Best performing model: {best_model_name} with mean accuracy {results[best_model_name]['mean_acc']:.4f}")

### Model Training and Evaluation Summary

#### Dataset
- **Train set size**: 176020 images
- **Test set size**: 44005 images
- The dataset consists of two classes. Data augmentation and preprocessing were applied to ensure robust training.

#### Model Architectures and Learning Rates
- **Custom CNN**: Learning rate of 0.0001
- **ResNet50**: Learning rate of 1e-05
- **EfficientNetB0**: Learning rate of 0.0001

#### Results

##### 1. Custom CNN
![Custom CNN Summary](visualizations/CNN_summary.png)

The Custom CNN was trained over 5 epochs on two different folds. Below are the key metrics:
- **Fold 1**: 
  - **Training Accuracy**: 0.8189 | **Validation Accuracy**: 0.8272
  - **Training AUC**: 0.8867 | **Validation AUC**: 0.8954
  - **Training Precision**: 0.8059 | **Validation Precision**: 0.7965
  - **Training Recall**: 0.7281 | **Validation Recall**: 0.7662
- **Fold 2**: 
  - **Training Accuracy**: 0.8134 | **Validation Accuracy**: 0.8103
  - **Training AUC**: 0.8797 | **Validation AUC**: 0.8827
  - **Training Precision**: 0.7985 | **Validation Precision**: 0.7741
  - **Training Recall**: 0.7215 | **Validation Recall**: 0.7544

- **Mean Accuracy**: 0.8133 (+/- 0.0090)

- **Key Findings**: 
  The Custom CNN demonstrated strong performance across both folds, achieving a mean accuracy of 81.33%. The model showed good balance between precision and recall, with slightly higher precision on the validation sets.

##### 2. ResNet50
![ResNet50 Summary](visualizations/resnet_summary.png)

The ResNet50 model was trained over 5 epochs on two different folds. Below are the key metrics:
- **Fold 1**: 
  - **Training Accuracy**: 0.7876 | **Validation Accuracy**: 0.7996
  - **Training AUC**: 0.8536 | **Validation AUC**: 0.8663
  - **Training Precision**: 0.7568 | **Validation Precision**: 0.7654
  - **Training Recall**: 0.7008 | **Validation Recall**: 0.7234
- **Fold 2**: 
  - **Training Accuracy**: 0.7893 | **Validation Accuracy**: 0.8002
  - **Training AUC**: 0.8542 | **Validation AUC**: 0.8696
  - **Training Precision**: 0.7596 | **Validation Precision**: 0.7841
  - **Training Recall**: 0.7019 | **Validation Recall**: 0.7031

- **Mean Accuracy**: 0.7935 (+/- 0.0027)

- **Key Findings**: 
  ResNet50 demonstrated stable performance with a mean accuracy of 79.35%. The model achieved strong AUC-ROC scores, indicating good separability between the two classes. However, the precision-recall tradeoff was not as balanced as the Custom CNN.

##### 3. EfficientNetB0
![EfficientNetB0 Summary](visualizations/efficientnet_summary.png)

The EfficientNetB0 model was trained over 5 epochs on two different folds. Below are the key metrics:
- **Fold 1**: 
  - **Training Accuracy**: 0.7978 | **Validation Accuracy**: 0.7935
  - **Training AUC**: 0.8669 | **Validation AUC**: 0.8665
  - **Training Precision**: 0.7723 | **Validation Precision**: 0.7351
  - **Training Recall**: 0.7102 | **Validation Recall**: 0.7610
- **Fold 2**: 
  - **Training Accuracy**: 0.7993 | **Validation Accuracy**: 0.7955
  - **Training AUC**: 0.8686 | **Validation AUC**: 0.8663
  - **Training Precision**: 0.7745 | **Validation Precision**: 0.7527
  - **Training Recall**: 0.7116 | **Validation Recall**: 0.7418

- **Mean Accuracy**: 0.7857 (+/- 0.0036)

- **Key Findings**: 
  EfficientNetB0 provided slightly lower accuracy compared to ResNet50 but performed comparably in terms of AUC. Precision and recall were well-balanced, indicating strong performance across both metrics. However, validation accuracy did not exceed 80%.

#### Conclusions and Findings
![Model Comparison](visualizations/model_comparison_mean_accuracy.png)

- **Best Performing Model**: The **Custom CNN** achieved the highest overall accuracy and AUC, with a mean accuracy of **81.33%** and mean AUC-ROC of **0.8954**.
- **ResNet50** provided competitive results but had slightly lower precision-recall balance compared to the Custom CNN.
- **EfficientNetB0** demonstrated good overall performance, though its accuracy did not surpass that of the Custom CNN or ResNet50.

In conclusion, while all three models performed well on this classification task, the **Custom CNN** exhibited the best balance between accuracy, precision, and recall, making it the top choice for this specific dataset and task.

### Hyperparameter Tuning and Final Model Training

#### Overview
This code block focuses on the hyperparameter tuning of the **Custom CNN** model and training the final model using the optimal hyperparameters. It builds upon the previous model evaluations and the successful use of preprocessed data, especially from the `train_preprocessed/` directory, where images were **cropped to the center 32x32 of the original 96x96 size** and then **enhanced using a super-resolution method** to resize them to 224x224 pixels.

#### Components from Previous Code Blocks

1. **Data**: 
   - Utilizes the `train_preprocessed/` images that were processed by cropping the center of the original images and then using a resolution increase function based on ESRGAN to resize them to 224x224. 
   - `train_labels` is used to provide labels for training, while `PREPROCESSED_TRAIN_PATH` points to the location of the preprocessed images.

2. **Functions**:
   - `create_generators()`: Used to create data generators for training and validation, ensuring consistent batch processing and augmentation.
   - `custom_cnn_preprocess()`: Used for image preprocessing, including normalization and any additional augmentations.
   - `plot_learning_curves()`: Visualizes the model's performance metrics over epochs.

#### New Functions and Classes

1. **F1ScoreMetric**:
   - A custom Keras metric class to compute the **F1 score** during training and validation, providing a crucial balance between precision and recall.

2. **PrintValMetricsCallback**:
   - A callback that prints key validation metrics (loss, accuracy, and F1 score) at the end of each epoch, giving insights into model performance after each training cycle.

3. **build_custom_cnn_model(hp)**:
   - This function builds a **Custom CNN** model for **hyperparameter tuning**, allowing flexible tuning of:
     - The number of convolutional and dense layers
     - The number of filters, units, kernel sizes
     - Dropout rates and learning rate
   - It integrates with Keras Tuner for optimal architecture search based on the training data.

#### Main Process

1. **Data Preparation**:
   - The dataset is split into `train_tune` and `val_tune` for hyperparameter tuning, using the **preprocessed images** to ensure consistent input size and quality.
   - **train_preprocessed/** images are used throughout, which were first cropped and then enhanced using the ESRGAN super-resolution method, improving the training effectiveness on higher resolution inputs.

2. **Hyperparameter Tuning**:
   - Utilizes **Keras Tuner's Hyperband algorithm** to search for optimal hyperparameters, with the objective of maximizing validation **F1-score**.
   - The **TqdmProgressCallback** provides a visual progress bar during training.

3. **Best Model Selection**:
   - The best hyperparameters are retrieved after the tuning process, optimizing the Custom CNN's architecture based on **AUC**, **F1-score**, and other metrics.

4. **K-fold Cross-Validation**:
   - **Stratified K-fold cross-validation** (3 folds) is performed to ensure robust model evaluation, providing insights into the model's generalization ability across different folds.

5. **Final Model Training**:
   - The final model is trained using the entire dataset after tuning, incorporating **early stopping** and **learning rate scheduling** to prevent overfitting.
   - The final model, including its weights, is saved for future evaluation and deployment.

6. **Model Saving and Evaluation**:
   - The best performing model is saved as `final_custom_cnn_model_1.keras`.
   - **Learning curves** are plotted to visualize the model's training and validation performance, showing trends in accuracy, AUC, precision, recall, and F1-score.

#### Accomplishments

1. **Preprocessing**: Utilized advanced preprocessing methods, including **cropping the center 32x32 pixels of the images** and applying **super-resolution** techniques to upsample the images to 224x224 for improved input quality.
2. **Hyperparameter Optimization**: Explored a wide range of model architectures and hyperparameters to optimize the Custom CNN.
3. **Cross-Validation**: Performed thorough cross-validation, ensuring that the model generalizes well across different subsets of the data.
4. **Performance Visualization**: Plotted the learning curves and final validation metrics, providing insights into model performance over epochs.

This code block takes the success of the Custom CNN from previous steps and systematically refines it through hyperparameter optimization and preprocessing improvements. The preprocessing steps, particularly the image resolution enhancement, provided a solid foundation for improved performance.

In [ ]:
# Ensure correct path for preprocessed data is used
PREPROCESSED_TRAIN_PATH = 'train_preprocessed/'

# Convert labels to strings
train_labels['label'] = train_labels['label'].astype(str)

# Define a custom F1 score metric as a class
class F1ScoreMetric(Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1ScoreMetric, self).__init__(name=name, **kwargs)
        self.precision = tf.keras.metrics.Precision()
        self.recall = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred, sample_weight)
        self.recall.update_state(y_true, y_pred, sample_weight)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        f1_val = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        return f1_val

    def reset_states(self):
        self.precision.reset_states()
        self.recall.reset_states()

# Callback to print validation metrics after each epoch
class PrintValMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        val_accuracy = logs.get('val_accuracy')
        val_f1 = logs.get('val_f1_score')  # F1-Score
        if val_loss is not None and val_accuracy is not None:
            print(f"\nEpoch {epoch + 1} - val_loss: {val_loss:.4f} - val_accuracy: {val_accuracy:.4f} - val_f1: {val_f1:.4f}")

# Build CNN model for hyperparameter tuning using Keras Tuner
def build_custom_cnn_model(hp):
    model = models.Sequential()
    model.add(layers.Input(shape=(224, 224, 3)))
    
    # Hyperparameter tuning for convolution layers
    for i in range(hp.Int('num_conv_layers', 2, 4)):  # Expanded range to 2-4 layers
        model.add(layers.Conv2D(
            filters=hp.Choice(f'conv_{i}_filters', [32, 64, 128, 256]),  # Added 256 filters for exploration
            kernel_size=hp.Choice(f'conv_{i}_kernel', [3, 5, 7]),  # Added kernel size 7
            activation='relu'
        ))
        model.add(layers.BatchNormalization())
        model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.GlobalAveragePooling2D())
    
    # Dense layer
    model.add(layers.Dense(
        units=hp.Int('dense_0_units', 64, 512, step=64),  # Expanded dense units to 512
        activation='relu'
    ))
    model.add(layers.Dropout(hp.Float('dropout_0', 0.2, 0.6)))  # Increased range for Dropout
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    # Compile model with binary cross-entropy and tuned learning rate
    model.compile(
        optimizer=Adam(hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),  # Increased upper limit of learning rate
        loss='binary_crossentropy',  # Replaced focal loss with binary cross-entropy
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                 tf.keras.metrics.Precision(name='precision'),
                 tf.keras.metrics.Recall(name='recall'),
                 F1ScoreMetric()]  # Custom F1 score metric
    )
    
    return model

# Update the hyperparameter tuner to monitor F1-score
tuner = kt.Hyperband(
    build_custom_cnn_model,
    objective=kt.Objective('val_f1_score', direction='max'),
    max_epochs=20,  # Increased max epochs to allow longer training
    factor=3,
    directory='hyperparameter_tuning',
    project_name='custom_cnn_tuning'
)

# Early stopping with F1-score
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_f1_score', patience=5, mode='max')

# Split the dataset and create training/validation generators
train_subset, _ = train_test_split(train_labels, test_size=0.8, stratify=train_labels['label'], random_state=42)
train_generator, val_generator, _, _ = create_generators(
    train_subset, train_subset.sample(frac=0.2, random_state=42), PREPROCESSED_TRAIN_PATH,
    batch_size=32,
    target_size=(224, 224),
    preprocess_func=custom_cnn_preprocess
)

# Hyperparameter tuning search
tuner.search(train_generator,
             epochs=10,
             validation_data=val_generator,
             callbacks=[stop_early, PrintValMetricsCallback()],
             verbose=1)

# Retrieve the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\nThe hyperparameter search is complete. The optimal hyperparameters are:")
for param, value in best_hps.values.items():
    print(f"{param}: {value}")

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)

# Perform K-fold Cross-Validation
n_splits = 3
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

cv_scores = []
histories = []

for fold, (train_idx, val_idx) in enumerate(skf.split(train_labels['id'], train_labels['label']), 1):
    print(f"\nFold {fold}/{n_splits}")
    
    fold_train_data = train_labels.iloc[train_idx].reset_index(drop=True)
    fold_val_data = train_labels.iloc[val_idx].reset_index(drop=True)
    
    # Preprocessed images for training
    fold_train_generator, fold_val_generator, _, _ = create_generators(
        fold_train_data, fold_val_data, PREPROCESSED_TRAIN_PATH,  
        batch_size=32,
        target_size=(224, 224),
        preprocess_func=custom_cnn_preprocess
    )
    
    fold_history = final_model.fit(
        fold_train_generator,
        epochs=15,
        validation_data=fold_val_generator,
        callbacks=[
            EarlyStopping(monitor='val_f1_score', patience=5, mode='max', restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_f1_score', factor=0.2, patience=3, min_lr=1e-6, mode='max'),
            PrintValMetricsCallback()
        ],
        verbose=1
    )
    
    fold_val_metrics = final_model.evaluate(fold_val_generator)
    cv_scores.append(fold_val_metrics)
    histories.append(fold_history)
    
    print(f"Fold {fold} validation metrics:")
    for name, value in zip(final_model.metrics_names, fold_val_metrics):
        print(f"{name}: {value:.4f}")

# Cross-validation results
mean_scores = np.mean(cv_scores, axis=0)
std_scores = np.std(cv_scores, axis=0)
for i, metric_name in enumerate(final_model.metrics_names):
    print(f"{metric_name}: {mean_scores[i]:.4f} (+/- {std_scores[i]:.4f})")

# Final train-validation split for training on the entire dataset
validation_split = 0.05
train_data, val_data = train_test_split(train_labels, test_size=validation_split, stratify=train_labels['label'], random_state=42)

# Preprocessed final training data
final_train_generator, final_val_generator, _, _ = create_generators(
    train_data, val_data, PREPROCESSED_TRAIN_PATH,  
    batch_size=32,
    target_size=(224, 224),
    preprocess_func=custom_cnn_preprocess
)

callbacks = [
    EarlyStopping(monitor='val_f1_score', patience=5, mode='max', restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_f1_score', factor=0.2, patience=3, min_lr=1e-6, mode='max'),
    tf.keras.callbacks.ModelCheckpoint("best_custom_cnn_model.keras", save_best_only=True, monitor='val_f1_score', mode='max'),
    PrintValMetricsCallback()
]

# Train the final model
history = final_model.fit(
    final_train_generator,
    validation_data=final_val_generator,
    epochs=30,
    callbacks=callbacks,
    verbose=1
)

# Save the final model
final_model.save("final_custom_cnn_model_1.keras")
print("Final model saved as 'final_custom_cnn_model_1.keras'")

# Updated plot_learning_curves function
def plot_learning_curves(history, model_name):
    metrics = ['loss', 'accuracy', 'auc', 'precision', 'recall', 'f1_score_metric']  # List of metrics to plot
    num_metrics = len(metrics)
    fig, axes = plt.subplots(num_metrics, 1, figsize=(10, 5*num_metrics))
    
    for i, metric in enumerate(metrics):
        if metric in history.history:
            axes[i].plot(history.history[metric], label=f'Train {metric}')
        if f'val_{metric}' in history.history:
            axes[i].plot(history.history[f'val_{metric}'], label=f'Validation {metric}')
        
        axes[i].set_title(f'{model_name} {metric.capitalize()}')
        axes[i].set_xlabel('Epoch')
        axes[i].set_ylabel(metric.capitalize())
        axes[i].legend()
        axes[i].grid(True)

    plt.tight_layout()
    plt.show()

# Plot the learning curves
plot_learning_curves(history, "Final Custom CNN")

### Hyperparameter Tuning Results

I performed hyperparameter tuning using Keras Tuner with the Hyperband algorithm to optimize our CNN model for cancer cell detection. The tuning process explored various architectural and training parameters to maximize the validation F1-Score, which ensures a balance between precision and recall.

#### Tuning Process Overview
- **Total Trials**: 26
- **Total Time**: 12 hours 45 minutes 44 seconds
- **Best Validation F1-Score**: 0.7802212834358215

#### Best Hyperparameters
The optimal configuration found by the tuner (Trial ID: 0026) is as follows:

1. **Model Architecture**:
   - **Number of Convolutional Layers**: 4
     - **Layer 1**: 32 filters, 5x5 kernel
     - **Layer 2**: 128 filters, 3x3 kernel
     - **Layer 3**: 256 filters, 7x7 kernel
     - **Layer 4**: 256 filters, 5x5 kernel
   - **Dense Layers**: 1
     - **Units**: 448
   - **Dropout Rate**: 0.5382

2. **Training Parameters**:
   - **Learning Rate**: 0.00029325587074452084
   - **Epochs**: 20

#### Performance Metrics
For the best trial (Trial ID: 0026), we observed the following metrics:

- **Training Accuracy**: 0.8603
- **Training AUC**: 0.9258
- **Training F1-Score**: 0.8203
- **Training Loss**: 0.3314
- **Training Precision**: 0.8581
- **Training Recall**: 0.7857
- **Validation Accuracy**: 0.8528
- **Validation AUC**: 0.9272
- **Validation F1-Score**: 0.7951
- **Validation Loss**: 0.3426
- **Validation Precision**: 0.9117
- **Validation Recall**: 0.7049

#### Analysis
The tuning process identified a model configuration that achieves a balanced validation F1-Score of 0.7951, ensuring both high precision and recall for cancer cell detection. The architecture, consisting of four convolutional layers with increasing complexity, followed by a single dense layer, appears to effectively capture the relevant features of the images.

The optimal learning rate (~ 2.93e-4) indicates a relatively careful and controlled optimization process, which is crucial for avoiding overfitting in complex models like CNNs.

The model demonstrates strong performance in precision (0.9117), meaning it effectively minimizes false positives, while maintaining a good recall (0.7049), indicating a decent ability to detect true positive cases. The performance suggests a good generalization to unseen data, which is critical in medical applications like cancer detection.

The overall training and validation metrics suggest that the model is not overfitting, and it maintains robust performance across both datasets. This optimal configuration will now be used for cross-validation and final testing on held-out data to further assess its effectiveness.

### Cross-Validation Results Summary

I performed **3-fold cross-validation** on our custom CNN model for cancer cell detection, with each fold training for 15 epochs. The cross-validation process took approximately **20-25 minutes per epoch** across all 15 epochs per fold, resulting in a **total runtime of about 15-18 hours** for the entire cross-validation process.

#### Dataset Information
- **Total images**: 220,025
- **Images per fold**: ~146,683 (training) and ~73,341 (validation)
- **Image dimensions**: 224x224x3 (RGB)
- **Batch size**: 32

#### Model Architecture
The model used is a custom CNN with the following key features:
- **4 convolutional layers**
  - Layer 1: 32 filters, 5x5 kernel
  - Layer 2: 128 filters, 3x3 kernel
  - Layer 3: 256 filters, 7x7 kernel
  - Layer 4: 256 filters, 5x5 kernel
- **1 dense layer with 448 units**
- **Dropout rate**: 0.5382
- **Initial learning rate**: 0.00029326

### Cross-Validation Results

#### Fold 1
- **Final Validation Metrics**:
  - Loss: 0.3426
  - Accuracy: 0.8528
  - AUC: 0.9272
  - Precision: 0.9117
  - Recall: 0.7049
  - F1-Score: 0.7951

#### Fold 2
- **Final Validation Metrics**:
  - Loss: 0.3057
  - Accuracy: 0.8713
  - AUC: 0.9369
  - Precision: 0.8757
  - Recall: 0.7951
  - F1-Score: 0.8334

#### Fold 3
- **Final Validation Metrics**:
  - Loss: 0.2954
  - Accuracy: 0.8782
  - AUC: 0.9417
  - Precision: 0.8940
  - Recall: 0.7933
  - F1-Score: 0.8406

### Overall Performance
- **Mean Loss**: 0.3146 (±0.0202)
- **Mean Accuracy**: 0.8674 (±0.0107)
- **Mean AUC**: 0.9353 (±0.0067)
- **Mean Precision**: 0.8938 (±0.0144)
- **Mean Recall**: 0.7644 (±0.0411)
- **Mean F1-Score**: 0.8227 (±0.0192)

#### Training Observations
1. **Consistent performance** across all folds with accuracies ranging between 85% and 87.8%.
2. The model achieved **high AUC scores (>0.92)** in all folds, showing strong discriminative ability.
3. **Precision was consistently high** across folds (>0.87), indicating effective identification of true positives.
4. **Recall showed some variability** between folds (0.7049 to 0.7951), which can potentially be addressed through threshold tuning or further refinement.
5. **Cross-validation took about 20-25 minutes per epoch**, resulting in a total time of approximately 15-18 hours across all 3 folds and 15 epochs per fold.
6. The learning rate progressively decreased during training, allowing for more precise adjustments to the weights.
7. The model displayed good generalization, with validation loss closely matching training loss in each fold.

#### Conclusion
The cross-validation results demonstrate that our custom CNN model performs robustly across multiple data splits. The **high precision and AUC scores** indicate strong potential for use in cancer cell detection tasks. While **recall variability** suggests an area for further tuning, the overall results show excellent generalization capabilities, making the model well-suited for real-world applications.

### Final Model Training Summary

The final custom CNN model was trained with **early stopping**, which halted the training at the **12th epoch** to prevent overfitting. The model was trained on a dataset containing **209,023 images** for training and **11,002 images** for validation, with a batch size of 32. Each epoch took approximately **30-35 minutes** to complete, resulting in a total training time of around **6-7 hours**.

#### Dataset Information
- **Training images**: 209,023
- **Validation images**: 11,002
- **Image dimensions**: 224x224x3 (RGB)
- **Batch size**: 32

#### Model Architecture
The model architecture, as determined by hyperparameter tuning, includes:
- **4 convolutional layers**:
  - Layer 1: 32 filters, 5x5 kernel
  - Layer 2: 128 filters, 3x3 kernel
  - Layer 3: 256 filters, 7x7 kernel
  - Layer 4: 256 filters, 5x5 kernel
- **1 dense layer with 448 units**
- **Dropout rate**: 0.5382
- **Initial learning rate**: 0.00029326 (decaying over the epochs)

### Training Results

![Final Model Training Metrics](visualizations/final_model_training_metrics.png)

#### Final Epoch (Epoch 12) Results
- **Training Accuracy**: 0.8640
- **Training AUC**: 0.9302
- **Training F1-Score**: 0.8246
- **Training Loss**: 0.3211
- **Training Precision**: 0.8658
- **Training Recall**: 0.7871

#### Validation Results
- **Validation Loss**: 0.3114
- **Validation Accuracy**: 0.8685
- **Validation AUC**: 0.9357
- **Validation F1-Score**: 0.8273
- **Validation Precision**: 0.8835
- **Validation Recall**: 0.7778

#### Training Observations
1. **Early stopping** triggered after the **12th epoch**, indicating that further training would not have resulted in significant improvements.
2. The model achieved **consistent validation metrics**, with **accuracy stabilizing around 86.8%** and the **F1-score around 0.827**.
3. The **AUC score remained high** at 0.9357, demonstrating excellent discriminative capability.
4. **Precision and recall** were well balanced, with precision at 0.8835 and recall slightly lower at 0.7778, suggesting good performance in identifying positive cases, but with room for further optimization through threshold tuning.

#### Conclusion
The model's performance stabilized by the 12th epoch, where **early stopping** prevented unnecessary additional training. The **strong accuracy, AUC, and F1-scores** indicate that the final model is well-suited for cancer cell detection tasks. It has been saved as **`final_custom_cnn_model.keras`** for future evaluation and deployment.

## Total Time Summary

### 1. Hyperparameter Tuning
- **Total Trials**: 26
- **Total Time**: ~12 hours 45 minutes
- **Best Validation F1-Score**: 0.7802

### 2. Cross-Validation
- **Number of Folds**: 3
- **Epochs per Fold**: 15
- **Time per Epoch**: ~20-25 minutes
- **Total Time for Cross-Validation**: ~18-20 hours

### 3. Final Model Training
- **Epochs Trained**: 12 (due to early stopping)
- **Time per Epoch**: ~30-35 minutes
- **Total Time for Final Model Training**: ~6-7 hours

### Overall Total Time: ~37-40 hours


In [ ]:
# Function to tune threshold after training
def tune_threshold(model, val_generator):
    # Get predictions (probabilities) and true labels
    val_generator.reset()  # Reset generator for proper evaluation
    y_true = []
    y_pred_proba = []

    # Reuse the val_generator from previous code blocks
    for i in range(len(val_generator)):
        x, y = val_generator[i]
        y_true.extend(y)
        y_pred_proba.extend(model.predict(x).flatten())

    y_true = np.array(y_true).astype(int)  # Ensure labels are integers
    y_pred_proba = np.array(y_pred_proba)

    # Precision-Recall curve to find optimal threshold
    precision, recall, thresholds = precision_recall_curve(y_true, y_pred_proba)
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)  # Avoid division by zero

    # Find the threshold that maximizes the F1-score
    optimal_idx = np.argmax(f1_scores)
    if thresholds.size == 0:  # Handle case where no thresholds are found
        print("No thresholds found, defaulting to 0.5")
        optimal_threshold = 0.5
    else:
        optimal_threshold = thresholds[optimal_idx]
    print(f"Optimal Threshold: {optimal_threshold}")

    # Apply the optimal threshold to get binary predictions
    y_pred = (y_pred_proba >= optimal_threshold).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision_metric = precision_score(y_true, y_pred)
    recall_metric = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred_proba)
    cm = confusion_matrix(y_true, y_pred)

    # Print results
    print("\nModel Performance Metrics with Optimal Threshold:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision_metric:.4f}")
    print(f"Recall: {recall_metric:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC-ROC: {auc_roc:.4f}")
    print("\nConfusion Matrix:")
    print(cm)

    return optimal_threshold, y_true, y_pred_proba

# Function to plot the optimized threshold metrics
def plot_optimized_threshold_metrics(y_true, y_pred_proba, optimal_threshold, model_name):
    # Apply the optimal threshold to get binary predictions
    y_pred = (y_pred_proba >= optimal_threshold).astype(int)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision_metric = precision_score(y_true, y_pred)
    recall_metric = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc_roc = roc_auc_score(y_true, y_pred_proba)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Create a plot
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC-ROC']
    values = [accuracy, precision_metric, recall_metric, f1, auc_roc]

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.barh(metrics, values, color='skyblue')
    ax.set_xlabel('Score')
    ax.set_title(f'Performance Metrics with Optimized Threshold ({model_name})')

    for i, v in enumerate(values):
        ax.text(v + 0.01, i, f"{v:.4f}", color='blue', va='center')

    plt.show()

    # Display confusion matrix
    print("\nConfusion Matrix:")
    print(cm)

    # Return metrics for future use or logging
    return {
        'accuracy': accuracy,
        'precision': precision_metric,
        'recall': recall_metric,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'confusion_matrix': cm
    }

# Reuse 'final_model' and 'final_val_generator' from earlier blocks
optimal_threshold, y_true, y_pred_proba = tune_threshold(final_model, final_val_generator)

# Plot the metrics after applying the optimal threshold
metrics = plot_optimized_threshold_metrics(y_true, y_pred_proba, optimal_threshold, model_name="Final Custom CNN")

### Performance Summary with Optimized Threshold

After training the final custom CNN model, I applied a threshold tuning procedure to optimize the F1-score. The optimal threshold was found to be **0.4179**, which was used to adjust the binary classification decision boundary. This threshold tuning significantly improved the model's performance across key metrics:

![Optimal Threshold Metrics](visualizations/optimal_threshold.png)

- **Accuracy**: 0.8689 (Improved from 0.8671)  
  The model now correctly classifies **86.89%** of the samples, a modest improvement after threshold tuning.
  
- **Precision**: 0.8495 (Improved from 0.8441)  
  Precision increased, showing fewer false positives and more accurate identification of positive cases.
  
- **Recall**: 0.8220 (Improved from 0.7778)  
  A notable improvement in recall indicates that the model is now able to identify a larger portion of the true positive cases.
  
- **F1-Score**: 0.8355 (Improved from 0.8273)  
  The F1-score, which balances precision and recall, increased from **82.73%** to **83.55%**, confirming the positive impact of the threshold adjustment.
  
- **AUC-ROC**: 0.9359 (Improved from 0.9357)  
  The AUC-ROC remains high, with a slight improvement, further demonstrating the model's ability to distinguish between positive and negative classes effectively.

Additionally, the confusion matrix shows the following:
- **True Negatives**: 5897
- **False Positives**: 649
- **False Negatives**: 793
- **True Positives**: 3663

The optimized threshold significantly boosted both recall and precision, leading to a better balance between these metrics. This demonstrates the effectiveness of threshold tuning in improving the model's ability to identify more true positives without overly increasing false positives. The result is a well-calibrated model with enhanced performance for both accuracy and class balance.


## Making Predictions and Kaggle Submission
Finally, let's use our best model to make predictions on the test set and create a submission file for Kaggle.

In [ ]:
# Load the saved model
# final_model = load_model("final_custom_cnn_model.keras")
print(f"Model summary:")
final_model.summary()

print(f"\nModel input shape: {final_model.input_shape}")
print(f"Model output shape: {final_model.output_shape}")

# Use the preprocessed test images
TEST_PATH = 'test_preprocessed/'
print(f"Test data directory: {TEST_PATH}")

# Create a DataFrame for test data
test_files = [f for f in os.listdir(TEST_PATH) if f.endswith('.tif')]
test_df = pd.DataFrame({'id': [f.split('.')[0] for f in test_files]})
print(f"Number of test samples: {len(test_df)}")

# Function to create test generator
def create_test_generator(test_data, test_path, batch_size, target_size=(224, 224), preprocess_func=None):
    test_data = test_data.copy()
    
    # Ensure filenames end with '.tif'
    test_data['id'] = test_data['id'].apply(lambda x: x if x.endswith('.tif') else x + '.tif')
    
    # Create ImageDataGenerator for test data
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_func)
    
    # Create generator for test data
    test_generator = test_datagen.flow_from_dataframe(
        test_data,
        directory=test_path,
        x_col='id',
        y_col=None,
        target_size=target_size,
        batch_size=batch_size,
        class_mode=None,
        shuffle=False
    )
    
    return test_generator

# Create test generator
test_generator = create_test_generator(
    test_df,
    TEST_PATH,
    batch_size=32,
    target_size=(224, 224),
    preprocess_func=custom_cnn_preprocess
)

# Make predictions on test data
print("Making predictions on test data...")
predictions = []
for i in range(len(test_generator)):
    batch = next(test_generator)
    batch = np.array(batch)  # Ensure the batch is a numpy array
    batch_predictions = final_model.predict(batch, verbose=0)
    predictions.extend(batch_predictions.flatten())
    if i % 10 == 0:
        print(f"Processed {i * 32}/{len(test_df)} images")

# **Dynamically use the optimal threshold from the previous block**
# Assuming 'optimal_threshold' was computed in the previous block using tune_threshold()
print(f"\nUsing the dynamically calculated optimal threshold: {optimal_threshold}")

# Convert probabilities to binary predictions using the optimal threshold
binary_predictions = [1 if pred >= optimal_threshold else 0 for pred in predictions]

# Create submission DataFrame
submission = pd.DataFrame({
    'id': test_df['id'],
    'label': binary_predictions
})

# Save the submission file
submission_filename = 'submission.csv'
submission.to_csv(submission_filename, index=False)
print(f"Submission file '{submission_filename}' created successfully!")

# Display the first few rows of the submission file
print("\nFirst few rows of the submission file:")
print(submission.head().to_string(index=False))

# Verify the format matches Kaggle's requirements
with open(submission_filename, 'r') as f:
    print("\nFirst few lines of the CSV file:")
    for i, line in enumerate(f):
        print(line.strip())
        if i == 5:  # Print first 5 lines
            break

# Check class distribution
class_distribution = submission['label'].value_counts(normalize=True)
print("\nClass Distribution:")
print(class_distribution)

# Plot the distribution of predictions
plt.figure(figsize=(10, 6))
plt.bar(class_distribution.index, class_distribution.values)
plt.title('Distribution of Predictions')
plt.xlabel('Class')
plt.ylabel('Proportion')
plt.xticks([0, 1])
plt.show()

# Additional checks
print(f"\nNumber of predictions: {len(submission)}")
print(f"Number of 0s: {(submission['label'] == 0).sum()}")
print(f"Number of 1s: {(submission['label'] == 1).sum()}")

print(f"\nYour submission file '{submission_filename}' is ready for upload to Kaggle!")

## Summary of Test Data Prediction and Submission Process

In this step, I performed predictions on a test dataset using the **final custom CNN model** and saved the results in a format suitable for submission to Kaggle. I used **preprocessed test images** that were created through a specific series of transformations, including **center cropping** and **super-resolution enhancement**.

1. **Model Summary**:  
   The loaded model is a custom CNN, consisting of convolutional, batch normalization, pooling, and fully connected layers, designed for binary classification. The total number of parameters in the model is **10,204,037**, with trainable parameters making up around **3.4 million**.

2. **Test Image Preprocessing**:  
   The test images were processed in the following way:
   - **Center Crop**: The central **32x32** region was cropped from the original **96x96** image to focus on the most important part of the image.
   - **Super-Resolution Model**: I applied an **ESRGAN (Enhanced Super-Resolution GAN)** model to upscale the cropped image to a **224x224** resolution. This process was implemented using a TensorFlow Hub ESRGAN model to enhance the detail and quality of the cropped images before feeding them into the model for prediction.

3. **Test Data Generator**:  
   I created a test generator to preprocess the test images in batches (32 images per batch). The generator loaded **57,458 test samples** from the preprocessed directory for prediction.

4. **Making Predictions**:  
   Using the pre-trained model, predictions were made on the test set. The predictions were generated as probabilities, which were then converted to binary classifications (0 or 1) using the dynamically calculated **optimal threshold** of **0.4179**.

5. **Submission Creation**:  
   A submission file was created in the required CSV format for Kaggle. The file contains two columns:
   - **id**: The ID of each test sample.
   - **label**: The predicted class (0 or 1), derived using the optimal threshold.

6. **Distribution of Predictions**:  
   A plot showing the distribution of predictions revealed that:
   - **63.2%** of the predictions were classified as **0**.
   - **36.8%** were classified as **1**.

![Prediction Distribution](visualizations/prediction_distribution.png)

7. **Class Distribution and Verification**:  
   The total number of predictions was **57,458**, with **36,311 samples** classified as **0** and **21,147 samples** classified as **1**. The submission file was saved as **'submission_test1.csv'**, ready for upload to Kaggle.

### Sample of Predictions

| Image ID                                   | Label |
|--------------------------------------------|-------|
| 054155c9e3206e565741be06102a1db2c23c31dc   | 0     |
| 7d9deca4ab1d007a08ef89198a362f9826b16c63   | 0     |
| 1d182790aeaf5f42b89f96ba865ed158b84e2b57   | 1     |
| 1a2ab9367eacb8c4e387b197997e9287c74cb934   | 1     |
| 5a74860a9c5c0a3e2577f793cbf2054cd1aeba08   | 0     |

### Sample Images with Labels

| Image                                                                   | Label |
|-------------------------------------------------------------------------|-------|
| ![Image 1](visualizations/054155c9e3206e565741be06102a1db2c23c31dc.png) | 0     |
| ![Image 2](visualizations/7d9deca4ab1d007a08ef89198a362f9826b16c63.png) | 0     |
| ![Image 3](visualizations/1d182790aeaf5f42b89f96ba865ed158b84e2b57.png) | 1     |
| ![Image 4](visualizations/1a2ab9367eacb8c4e387b197997e9287c74cb934.png) | 1     |
| ![Image 5](visualizations/5a74860a9c5c0a3e2577f793cbf2054cd1aeba08.png) | 0     |

### Class Distribution

| Label | Count  |
|-------|--------|
| 0     | 63.20% |
| 1     | 36.80% |

This step demonstrates the complete process of preparing and preprocessing the test data, making predictions using the custom CNN model, and creating the submission file. The use of ESRGAN for enhancing the image resolution ensures that the model receives high-quality inputs, which contributes to the accuracy and reliability of the predictions.


## Conclusion and Future Work

In conclusion, this project successfully demonstrated the ability to design a custom CNN model capable of detecting metastatic cancer in histopathologic images with high precision and recall. Through careful model design, hyperparameter tuning, and image preprocessing techniques, I was able to achieve strong performance metrics that rivaled pre-trained models like ResNet and EfficientNet. This highlights the effectiveness of custom CNN models tailored specifically to a given dataset.

### Future Work

Moving forward, several areas of improvement could be explored to further enhance the model’s performance:
- **Ensemble Models**: Combining my custom CNN with other pre-trained models like ResNet and EfficientNet through ensemble techniques could lead to higher accuracy and robustness.
- **Data Augmentation**: Applying more advanced data augmentation techniques could help the model generalize better by exposing it to more variations of the data.
- **Transfer Learning**: Integrating transfer learning by fine-tuning pre-trained models could yield better results, especially with larger, more complex datasets.
- **Explainability**: Incorporating techniques such as Grad-CAM or saliency maps could provide insights into what areas of the image the model focuses on, making it more interpretable for medical use.

By addressing these points, this model could become an even more reliable tool for assisting in the early detection of cancer, potentially improving diagnostic accuracy in medical practice.